In [44]:
import sympy as sp
import numpy as np
from sympy import diff
from IPython.display import display

In [55]:
from commons import Surface,MethodOfOrthonormalFrames

In [25]:
u,v = sp.symbols("u,v")
du,dv=sp.symbols("du,dv")

In [5]:
#We choose such paramethrization of X(u),Y(u) that:
# (dx/du)**2 + (dy/du)**2 = 1

def x(u):
    return (u+5)/sp.sqrt(2)

def y(u):
    return (-u-1)/sp.sqrt(2)

def z(v):
    return v

assert diff(x(u),u)**2+diff(y(u),u)**2 == 1

In [6]:
#The First fundamental form is given by: I = (du)**2+(dv)**2
#The Mean curvature H is not determined by the I fundamental form alone
#On the other hand, the Gaussian curvature K depends only on the I fundamental form

#Consider I = Edudu +2Fdudv + Gdvdv (3.1.4) in a domain D of the (u,v) plane
#When (3.1.4) is positive definite form, we call it a Riemann metrics on D

#Let s - a parameter which represents length.
# => Edudu/dsds +2Fdudv/dsds + Gdvdv/dsds = 1 <=>
# dsds = Edudu +2Fdudv + Gdvdv; ds**2 <=> a Riemann metrics

# Example 1

Poincarre metrics:

Let D = {(u,v) | u**2+v**2<1}

$\text{We call } ds^2 = 4\frac{(du)^2 +(dv)^2}{(1-u^2+v^2)^2} \text{ the }\textbf{Poincarre metrics}$

Let w = u + i*v (using the complex plane)


$\text{=> We can rewrite the Poincarre metrics as follows: }$
$$ds^2 = \frac{4dwd\overline{w}}{1-|w|^2}$$
$\text{Where } dw = du + idv, d\overline{w} = du − idv$

# Example 2

Let U = {(x,y) | y>0}

Let z = i+xy

The example below is equivalent to the Poincarre metrics:

$$ds^2 = \frac{(dx)^2 + (dy)^2}{y^2} \equiv \frac{dzd\overline{z}}{y^2}$$

Hint: use transformation $z = i\frac{1-w}{1+w}$
the inverse map: $w = \frac{i-z}{i+z}$

#### Problem 3.1.1 Compare the first and second fundamental forms, H, K, κ1, and κ2 of a catenoid

In [18]:
class Surface:
    def __init__(self,u,v,p:tuple):
        self.u=u
        self.v=v
        self.p=p
        
        self.p_u=[diff(x,self.u) for x in self.p]
        self.p_v=[diff(x,self.v) for x in self.p]
        
        self.p_uu=[diff(x,self.u) for x in self.p_u]
        self.p_uv=[diff(x,self.v) for x in self.p_u]
        self.p_vv=[diff(x,self.v) for x in self.p_v]
        
        E=np.dot(self.p_u,self.p_u)
        F=np.dot(self.p_u,self.p_v)
        G=np.dot(self.p_v,self.p_v)
        self.E=E
        self.F=F
        self.G=G
        
        e=np.cross(self.p_u,self.p_v)
        e_mod=sp.sqrt(np.dot(e,e))
        self.e=[(x/e_mod).simplify() for x in e]
        
        L=np.dot(self.p_uu,self.e)
        M=np.dot(self.p_uv,self.e)
        N=np.dot(self.p_vv,self.e)
        self.L=L
        self.M=M
        self.N=N
        self.K = (L*N-M**2)/(E*G-F**2)
        self.H = (E*N+G*L-2*F*M)/(2*E*G-2*F**2)
        self.K=self.K.simplify()
        self.H=self.H.simplify()
        k1,k2=sp.symbols("k1,k2")
        
        k1k2=sp.solve([
            k1*k2-self.K,
            2*(k1+k2)-self.H
        ],(k1,k2))
        self.k1=k1k2[0]
        self.k2=k1k2[1]
        
    def val(self,u,v):
        return tuple(x.subs(self.u,u).subs(self.v,v).evalf() for x in self.p)

    def e_val(self,u,v):
        return tuple(x.subs(self.u,u).subs(self.v,v).evalf() for x in self.e)
    
    def I(self,du,dv):
        return self.E*du*du + 2*self.F*du*dv + self.G*dv*dv
    
    def II(self,du,dv):
        return self.L*du*du + 2*self.M*du*dv + self.N*dv*dv
    
    def III(self,du,dv):
        return (-self.K*self.I(du,dv)+2*self.H*self.II(du,dv)).simplify()

In [26]:
# Problem 3.1.1 Compare the first and second fundamental forms, H, K, κ1, and κ2 of a catenoid
catenoid = Surface(u,v,[
    sp.sqrt(u**2+1)*sp.sin(v),
    sp.sqrt(u**2+1)*sp.cos(v),
    sp.asinh(u)
])
# with those of a right helicoid:
helicoid = Surface(u,v,[
    u*sp.cos(v),
    u*sp.sin(v),
    v
])

## I(u,v)

In [45]:
display(helicoid.I(du,dv))
display(catenoid.I(du,dv))

du**2*(sin(v)**2 + cos(v)**2) + dv**2*(u**2*sin(v)**2 + u**2*cos(v)**2 + 1)

du**2*(u**2*sin(v)**2/(u**2 + 1) + u**2*cos(v)**2/(u**2 + 1) + 1/(u**2 + 1)) + dv**2*((u**2 + 1)*sin(v)**2 + (u**2 + 1)*cos(v)**2)

## II(u,v)

In [46]:
display(helicoid.II(du,dv))
display(catenoid.II(du,dv))

du*dv*(-2*sin(v)**2/sqrt(u**2 + 1) - 2*cos(v)**2/sqrt(u**2 + 1))

du**2*(u**2/(u**2 + 1)**2 + (-u**2*sin(v)/(u**2 + 1)**(3/2) + sin(v)/sqrt(u**2 + 1))*sin(v)/sqrt(u**2 + 1) + (-u**2*cos(v)/(u**2 + 1)**(3/2) + cos(v)/sqrt(u**2 + 1))*cos(v)/sqrt(u**2 + 1)) + dv**2*(-sin(v)**2 - cos(v)**2)

## k1

In [54]:
display(sp.Array(helicoid.k1))
display(sp.Array(catenoid.k1))

Helicoid


[-1/(u**2 + 1), 1/(u**2 + 1)]

Catenoid


[-1/(u**2 + 1), 1/(u**2 + 1)]

## k2

In [48]:
display(sp.Array(helicoid.k2))
display(sp.Array(catenoid.k2))

[1/(u**2 + 1), -1/(u**2 + 1)]

[1/(u**2 + 1), -1/(u**2 + 1)]

## H

In [49]:
display(helicoid.H)
display(catenoid.H)

0

0

## K

In [50]:
display(helicoid.K)
display(catenoid.K)

-1/(u**2 + 1)**2

-1/(u**2 + 1)**2

In [64]:
h=MethodOfOrthonormalFrames(helicoid)
alpha,beta=h.calc_mat_W()
display(alpha)
display(beta)
display(h.calc_mat_A())
display(h.calc_mat_B())

Matrix([
[0,            0,             0],
[0,            0, -1/(u**2 + 1)],
[0, 1/(u**2 + 1),             0]])

Matrix([
[                0, u/sqrt(u**2 + 1), -1/sqrt(u**2 + 1)],
[-u/sqrt(u**2 + 1),                0,                 0],
[ 1/sqrt(u**2 + 1),                0,                 0]])

Matrix([
[1,              0],
[0, sqrt(u**2 + 1)]])

Matrix([
[            0, -1/(u**2 + 1)],
[-1/(u**2 + 1),             0]])

In [65]:
c=MethodOfOrthonormalFrames(catenoid)
alpha,beta=c.calc_mat_W()
display(alpha)
display(beta)
display(c.calc_mat_A())
display(c.calc_mat_B())

Matrix([
[            0, 0, 1/(u**2 + 1)],
[            0, 0,            0],
[-1/(u**2 + 1), 0,            0]])

Matrix([
[                0, u/sqrt(u**2 + 1),                 0],
[-u/sqrt(u**2 + 1),                0, -1/sqrt(u**2 + 1)],
[                0, 1/sqrt(u**2 + 1),                 0]])

Matrix([
[1,              0],
[0, sqrt(u**2 + 1)]])

Matrix([
[1/(u**2 + 1),             0],
[           0, -1/(u**2 + 1)]])

In [75]:
def display_frame(c):
    display(sp.Array(c.e1).simplify())
    display(sp.Array(c.e2).simplify())
    display(sp.Array(c.e3).simplify())
    
print("Catenoid")
display_frame(c)
print("\nHelicoid")
display_frame(h)


Catenoid


[u*sin(v)/sqrt(u**2 + 1), u*cos(v)/sqrt(u**2 + 1), 1/sqrt(u**2 + 1)]

[cos(v), -sin(v), 0]

[sin(v)/sqrt(u**2 + 1), cos(v)/sqrt(u**2 + 1), -u/sqrt(u**2 + 1)]


Helicoid


[cos(v), sin(v), 0]

[-u*sin(v)/sqrt(u**2 + 1), u*cos(v)/sqrt(u**2 + 1), 1/sqrt(u**2 + 1)]

[sin(v)/sqrt(u**2 + 1), -cos(v)/sqrt(u**2 + 1), u/sqrt(u**2 + 1)]